Project Documenation and Cost Management

Project Management and Documentation:

1. Manage code using Git and maintain version control.
2. Create a well-organized repository following the naming convention specied in the requirements
3. Document all code, decisions, and methodologies used throughout the project, the more the better...
4. Generate and analysis report covering EDA, data pre-processing, feature selection, model choice, evaluation metrics, model tuning, and performance monitoring



Cost Management:

Keep track of the resources used during the project, inlcuding compute resources fro training and deploying the model
Analyse the project cost and identify opportunities for optimization
Docuemtn the total cost to deploy the model, including any additional cost monitoring or resource optimization done